In [15]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle




def preProcessRow( row: dict ):
       
    row['age']=(row['age']-20)/124
    row['salary']=(row['salary']-4000)/5996000
    
    if row['home_ownership']=='RENT':
        row['home_ownership'] = 0.312350
    elif row['home_ownership']=='MORTGAGE':
        row['home_ownership'] = 0.125837 
    elif row['home_ownership']=='OWN':
        row['home_ownership'] = 0.066606
    else:
        row['home_ownership'] = 0.287234
        
    if row['loan_purposes']=='EDUCATION':
        row['loan_purposes'] = 0.170231
    elif row['loan_purposes']=='MEDICAL':
        row['loan_purposes'] = 0.268468 
    elif row['loan_purposes']=='HOMEIMPROVEMENT':
        row['loan_purposes'] = 0.256723
    elif row['loan_purposes']=='DEBTCONSOLIDATION':
        row['loan_purposes'] = 0.283899
    elif row['loan_purposes']=='VENTURE':
        row['loan_purposes'] = 0.146171    
    else:
        row['loan_purposes'] = 0.197457
        
    #row['Payment_History']=row['Payment_History'].apply(lambda value: 0 if value=='N' else 1)    
    
    row['salary']=np.sqrt(row['salary'])
    row['loan_rate']=np.power(row['loan_rate'], 2)
    row['loan_percentage']=np.cos(row['loan_percentage'])

    
    return row

In [ ]:
import uvicorn
from fastapi import FastAPI
import nest_asyncio
nest_asyncio.apply()
import random
import pickle
import json

app = FastAPI()

#! Load the model
# model = None
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)
    
@app.post("/predict")
async def predict(data: str):
    #! Try decoding data 
    data = json.loads(data)
    #! Preprocess the data

    row = preProcessRow( data )
    row = list(row.values())
    row = [float(r) for r in row]
    return {"result": int(model.predict( [row] )[0])}
    

uvicorn.run(app, host="0.0.0.0", port=5000)

INFO:     Started server process [9624]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59099 - "POST /predict?data={%22age%22:%2018,%20%22salary%22:%2017000,%20%22home_ownership%22:%20%22RENT%22,%20%22employment_time%22:%203.0,%20%22loan_purposes%22:%20%22EDUCATION%22,%20%20%22Credit_Amount%22:%2012000,%20%22loan_rate%22:%207.9,%20%20%22loan_percentage%22:%203} HTTP/1.1" 200 OK
